In [ ]:
from langchain_community.chat_models.huggingface import ChatHuggingFace

llm = HuggingFaceEndpoint(repo_id="TinyLlama/TinyLlama-1.1B-Chat-v1.0")

chat_model = ChatHuggingFace(llm=llm)


In [ ]:
from langchain import hub
from langchain.agents import AgentExecutor, load_tools
from langchain.agents.format_scratchpad import format_log_to_str
from langchain.agents.output_parsers import (
    ReActJsonSingleInputOutputParser,
)
from langchain.tools.render import render_text_description
from langchain_community.utilities import SerpAPIWrapper

# setup tools
tools = load_tools(["wikipedia", "llm-math"], llm=llm)

In [ ]:
ChatPromptTemplate(
    input_variables=['agent_scratchpad', 'input'],
    partial_variables={
        'tools': 'wikipedia - A wrapper around Wikipedia. Useful for when you need to answer general questions about people, places, companies, facts, historical events, or other subjects. Input should be a search query.\nCalculator(*args: Any, callbacks: Union[List[langchain_core.callbacks.base.BaseCallbackHandler], langchain_core.callbacks.base.BaseCallbackManager, NoneType] = None, tags: Optional[List[str]] = None, metadata: Optional[Dict[str, Any]] = None, **kwargs: Any) -> Any - Useful for when you need to answer questions about math.', 
        'tool_names': 'wikipedia, Calculator'
    }, 
    metadata={
        'lc_hub_owner': 'hwchase17', 
        'lc_hub_repo': 'react-json', 
        'lc_hub_commit_hash': '669cf4d6988c3b8994a8189edb3891e07948e1c0abfd500823914548c53afa7c'
    },
    messages=[
        SystemMessagePromptTemplate(
            prompt=PromptTemplate(
                input_variables=['tool_names', 'tools'], 
                template='Answer the following questions as best you can. You have access to the following tools:\n\n{tools}\n\nThe way you use the tools is by specifying a json blob.\nSpecifically, this json should have a `action` key (with the name of the tool to use) and a `action_input` key (with the input to the tool going here).\n\nThe only values that should be in the "action" field are: {tool_names}\n\nThe $JSON_BLOB should only contain a SINGLE action, do NOT return a list of multiple actions. Here is an example of a valid $JSON_BLOB:\n\n```\n{{\n  "action": $TOOL_NAME,\n  "action_input": $INPUT\n}}\n```\n\nALWAYS use the following format:\n\nQuestion: the input question you must answer\nThought: you should always think about what to do\nAction:\n```\n$JSON_BLOB\n```\nObservation: the result of the action\n... (this Thought/Action/Observation can repeat N times)\nThought: I now know the final answer\nFinal Answer: the final answer to the original input question\n\nBegin! Reminder to always use the exact characters `Final Answer` when responding.')
        ), 
        HumanMessagePromptTemplate(
            prompt=PromptTemplate(
                input_variables=['agent_scratchpad', 'input'],
                template='{input}\n\n{agent_scratchpad}')
        )
    ]
)


In [ ]:
from transformers import CodeAgent, HfEngine
from langchain.agents import load_tools

tool = load_tools(["llm-math"])

llm_engine = HfEngine(model="TinyLlama/TinyLlama-1.1B-Chat-v1.0")
agent = CodeAgent(tools=[tool], llm_engine=llm_engine, add_base_tools=True)

agent.run(
    "What is the 25% of 300?"
)